In [1]:
import pypsa
import pandas as pd


In [2]:
import warnings
warnings.filterwarnings("ignore")

## Add total cost

In [3]:
def rename_techs(label):
    if "H2" in label:
        label = "hydrogen storage"
    elif label == "solar":
        label = "solar PV"
    elif label == "offwind-ac":
        label = "offshore wind ac"
    elif label == "offwind-dc":
        label = "offshore wind dc"
    elif label == "onwind":
        label = "onshore wind"
    elif label == "ror":
        label = "hydroelectricity"
    elif label == "hydro":
        label = "hydroelectricity"
    elif label == "PHS":
        label = "hydroelectricity"
    elif "battery" in label:
        label = "battery storage"

    return label


preferred_order = pd.Index(
    [
        "transmission lines",
        "hydroelectricity",
        "hydro reservoir",
        "run of river",
        "pumped hydro storage",
        "onshore wind",
        "offshore wind ac",
        "offshore wind dc",
        "solar PV",
        "solar thermal",
        "OCGT",
        "hydrogen storage",
        "battery storage",
    ]
)

def summarize_system_cost(infn_costs):
    df_all = pd.DataFrame()
    
    for year in infn_costs:
        # For now ignore the simpl header
        cost_df = pd.read_csv(infn_costs[year], index_col=list(range(3)), header=[1, 2, 3])

        #cost_df.columns = [str(infn[fn])]
    
        df = cost_df.groupby(cost_df.index.get_level_values(2)).sum()

        # If there is any unique carrier to remove
        if "B2B" in df.index:
            df = df.drop("B2B")

        # convert to billions
        #df = df / 1e9
    
        df = df.groupby(df.index.map(rename_techs)).sum()

        df.columns = [year]
        
        if df_all.empty:
            df_all = df
        else:
            df_all = pd.concat([df_all,df],axis=1)

    return df_all

def summarize_capacity(infn_cap):
    df_all = pd.DataFrame()

    for year in infn_cap:
        # For now ignore the simpl header
        cap_df = pd.read_csv(infn_cap[year], index_col=list(range(2)), header=[1, 2, 3])

        cap_df["component"] = cap_df.index.get_level_values(0)

        cap_df = cap_df#[cap_df.component == "generators"]

        cap_df = cap_df.drop(columns=["component"])
    
        df = cap_df.groupby(cap_df.index.get_level_values(1)).sum()

        # If there is any unique carrier to remove
        if "B2B" in df.index:
            df = df.drop("B2B")

        # convert to kW to MW
        #df = df / 1e3
    
        df = df.groupby(df.index.map(rename_techs)).sum()

        df.columns = [year]
        
        if df_all.empty:
            df_all = df
        else:
            df_all = pd.concat([df_all,df],axis=1)

    return df_all

def summarize_carrier_cost(fn,columns):
    df_gen = pd.DataFrame(data={year: fn[year].generators.groupby("carrier")[columns].mean() for year in fn})
    df_store = pd.DataFrame(data={year: fn[year].stores.groupby("carrier")[columns].mean() for year in fn})
    if columns == "capital_cost":
        df_line = pd.DataFrame(data={year: fn[year].lines.groupby("carrier")[columns].mean() for year in fn}).rename(index={"AC":"lines"})
        df = pd.concat([df_gen,df_store,df_line])
        
    else:
        df = pd.concat([df_gen,df_store])
        
    df = df.groupby(df.index.map(rename_techs)).sum()
    return df

def readjust_system_cost(scenario_infn_cap, scenario_infn_energy, scenario_infn_costs, fn):
    df_costs_raw = summarize_system_cost(scenario_infn_costs)
    df_cap = summarize_capacity(scenario_infn_cap)
    df_capital = summarize_carrier_cost(fn,"capital_cost")
    
    df_costs_base = df_costs_raw - (df_cap * df_capital)
    
    df_cap_change = df_cap.copy(deep=True)
    df_costs_clean = df_costs_base.copy(deep=True)
    
    for i in range(0,len(years)):
        df_cap_change[years[i]] = df_cap[years[i]] - df_cap_change[years[:i]].T.sum().T
    
    df_cost_positive = df_cap_change.clip(lower=0) * df_capital
    df_cost_negative = (df_cap_change.clip(upper=0).T * df_capital[2020]).T
    
    df_cost_yearly = df_cost_positive + df_cost_negative
    
    for i in range(0,len(years)):
        df_costs_clean[years[i]] = df_costs_base[years[i]] + df_cost_yearly[years[:i+1]].T.sum().T

    return df_costs_clean

In [4]:
scenarios = ["SEA_EXIST_", "SEA_AIMS_", "SEA_IRENA_"]
folder_name = "pypsa-earth"
network_name_complete = {
                "BAU":{
                    2020:"elec_s_200_ec_lcopt_3H",
                    2025:"elec_s_200_ec_lcopt_3H",
                    2030:"elec_s_200_ec_lcopt_3H",
                    2035:"elec_s_200_ec_lcopt_3H",
                    2040:"elec_s_200_ec_lcopt_3H",
                    2045:"elec_s_200_ec_lcopt_3H",
                    2050:"elec_s_200_ec_lcopt_3H",
                },
                "SE":{
                    2020:"elec_s_200_ec_lcopt_Co2L1-3H",
                    2025:"elec_s_200_ec_lcopt_Co2L0.916821-3H",
                    2030:"elec_s_200_ec_lcopt_Co2L0.833641-3H",
                    2035:"elec_s_200_ec_lcopt_Co2L0.750462-3H",
                    2040:"elec_s_200_ec_lcopt_Co2L0.667283-3H",
                    2045:"elec_s_200_ec_lcopt_Co2L0.584104-3H",
                    2050:"elec_s_200_ec_lcopt_Co2L0.500924-3H",
                     },
                "DEC":{
                    2020:"elec_s_200_ec_lcopt_Co2L1-3H",
                    2025:"elec_s_200_ec_lcopt_Co2L0.779298-3H",
                    2030:"elec_s_200_ec_lcopt_Co2L0.583549-3H",
                    2035:"elec_s_200_ec_lcopt_Co2L0.412754-3H",
                    2040:"elec_s_200_ec_lcopt_Co2L0.266913-3H",
                    2045:"elec_s_200_ec_lcopt_Co2L0.146026-3H",
                    2050:"elec_s_200_ec_lcopt_Co2L0.050092-3H"
                     }
                }

years = [2020,2025,2030,2035,2040,2045,2050]
country = "all"

df_fn = pd.DataFrame(index=["BAU","SE","DEC"],columns=scenarios)

for dec_name in df_fn.index:
    for scenario in df_fn.columns:
        fn = {year:pypsa.Network(f"../{folder_name}/results/{scenario}{year}/networks/{network_name_complete[dec_name][year]}.nc") for year in years}
        scenario_infn_cap = {year:f"../{folder_name}/results/{scenario + str(year)}/summaries/{network_name_complete[dec_name][year]}_{country}/capacity.csv" for year in years}
        scenario_infn_energy = {year:f"../{folder_name}/results/{scenario + str(year)}/summaries/{network_name_complete[dec_name][year]}_{country}/energy.csv" for year in years}
        scenario_infn_costs = {year:f"../{folder_name}/results/{scenario + str(year)}/summaries/{network_name_complete[dec_name][year]}_{country}/costs.csv" for year in years}

        df_fn.loc[dec_name,scenario] = readjust_system_cost(scenario_infn_cap, scenario_infn_energy, scenario_infn_costs, fn)[2050].sum()

INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc

## Add all the other

In [56]:
def get_renewable_share(n):
    res_carrier = ["solar", "onwind", "offwind-ac", "offwind-dc", "hydro"]
    res_index = n.generators[n.generators.carrier.isin(res_carrier)].index
    
    res = (n.snapshot_weightings.generators @ n.generators_t.p[res_index]).sum()
    total = (n.snapshot_weightings.generators @ n.generators_t.p).sum()
    
    return round(res/total * 100,1)

def get_annual_emission(n):
    return ((n.snapshot_weightings.generators @ n.generators_t.p) / n.generators.efficiency * n.generators.carrier.map(n.carriers.co2_emissions)).sum()

def get_coal_capacity_factor(n):
    coal_carrier = ['Coal-Usc', 'Coal-Sc', 'Coal-Cfb', 'Coal-Subc', 'Coal', 'Lignite']

    cap_factor = n.statistics.capacity_factor()
    cap_factor = cap_factor.groupby(cap_factor.index.get_level_values(1)).sum()
    
    cap = n.statistics.optimal_capacity()
    cap = cap.groupby(cap.index.get_level_values(1)).sum()
    
    return round((cap_factor[coal_carrier] * cap[coal_carrier] / cap[coal_carrier].sum()).sum() * 100,1)

def get_total_transmission_expansion(n):
    return round(((n.lines.s_nom_opt * n.lines.length).sum() + (n.links.p_nom_opt * n.links.length).sum()) / 1e6,1) # MVAkm to TVAkm

def get_total_energy_traded(n):
    n.lines["country0"] = [n.buses.country[busname] for busname in n.lines.bus0]
    n.lines["country1"] = [n.buses.country[busname] for busname in n.lines.bus1]
    lines_index = n.lines.query('country0 != country1').index
    
    lines = (n.snapshot_weightings.generators @ abs(n.lines_t.p0[lines_index])).sum()
    
    n.links["country0"] = [n.buses.country[busname] for busname in n.links.bus0]
    n.links["country1"] = [n.buses.country[busname] for busname in n.links.bus1]
    links_index = n.links.query('country0 != country1').index
    
    links = (n.snapshot_weightings.generators @ abs(n.links_t.p0[links_index])).sum()
    
    return round((lines + links) / 1e6,1) # MWh to TWh

def get_total_demand(n):
    return (n.snapshot_weightings.generators @ n.loads_t.p_set).sum()

In [7]:
scenarios = ["SEA_EXIST_", "SEA_AIMS_", "SEA_IRENA_"]
folder_name = "pypsa-earth"
network_name = {
                "BAU":"elec_s_200_ec_lcopt_3H",
                "SE":"elec_s_200_ec_lcopt_Co2L0.500924-3H",
                "DEC":"elec_s_200_ec_lcopt_Co2L0.050092-3H"
                }

year = 2050

df = pd.DataFrame(index=["BAU","SE","DEC"],columns=scenarios)
for dec_name in df.index:
    for scenario in df.columns:
        df.loc[dec_name,scenario] = pypsa.Network(f"../{folder_name}/results/{scenario}{year}/networks/{network_name[dec_name]}.nc")

INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_3H.nc has buses, carriers, generators, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_Co2L0.500924-3H.nc has buses, carriers, generators, global_constraints, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_Co2L0.500924-3H.nc has buses, carriers, generators, global_constraints, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_Co2L0.500924-3H.nc has buses, carriers, generators, global_constraints, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_ec_lcopt_Co2L0.050092-3H.nc has buses, carriers, generators, global_constraints, lines, links, loads, stores
INFO:pypsa.io:Imported network elec_s_200_

In [61]:
df_base = pd.DataFrame(index=["BAU","SE","DEC"],columns=scenarios)
df_base.index = df_base.index.set_names(['Policy'])

df_res = df_base.copy(deep=True)
df_co2 = df_base.copy(deep=True)
df_ave_co2 = df_base.copy(deep=True)
df_coal = df_base.copy(deep=True)
df_trans = df_base.copy(deep=True)
df_trade = df_base.copy(deep=True)
df_system = df_base.copy(deep=True)
df_ave_cost = df_base.copy(deep=True)

for scenario in df.columns:
    df_res[scenario] = df[scenario].apply(lambda n: get_renewable_share(n))
    df_co2[scenario] = df[scenario].apply(lambda n: round(get_annual_emission(n)/1e6,1))
    df_ave_co2[scenario] = (df[scenario].apply(lambda n: get_annual_emission(n))/df[scenario].apply(lambda n: get_total_demand(n))).apply(lambda x: round(x*1e3,1))
    df_coal[scenario] = df[scenario].apply(lambda n: get_coal_capacity_factor(n))
    df_trans[scenario] = df[scenario].apply(lambda n: get_total_transmission_expansion(n))
    df_trade[scenario] = df[scenario].apply(lambda n: get_total_energy_traded(n))
    df_system[scenario] = df_fn[scenario].apply(lambda x: round(x / 1e9,1))
    df_ave_cost[scenario] = (df_fn[scenario]/df[scenario].apply(lambda n: get_total_demand(n))).apply(lambda x: round(x,1))

df_res["KPI"] = "Renewable Share [%]"
df_co2["KPI"] = "Annual Emission [MtCO2/a]"
df_ave_co2["KPI"]  = "Emission Intensity [gCO2/kWh]"
df_coal["KPI"] = "Coal Capacity Factor [%]"
df_trans["KPI"] = "Transmission Capacity [TVA km]"
df_trade["KPI"] = "Energy Trade [TWh]"
df_system["KPI"] = "Annual System Cost [EUR Bil/a]"
df_ave_cost["KPI"] = "Marginal Price [EUR/MWh]"

df_all = pd.concat([df_res, df_co2, df_ave_co2, df_coal, df_trans, df_trade, df_system, df_ave_cost])
df_all = df_all.reset_index()
df_all = df_all.reset_index()
df_all = df_all.groupby(["KPI","Policy"]).sum()
df_all = df_all.sort_values("index").drop(["index"], axis=1)

In [62]:
df_all

SEA_EXIST_  SEA_AIMS_  SEA_IRENA_
KPI                            Policy                                   
Renewable Share [%]            BAU           60.2       63.8        65.7
                               SE            70.4       74.1        75.6
                               DEC           90.8       90.5        90.6
Annual Emission [MtCO2/a]      BAU         1074.7      995.3       956.6
                               SE           505.4      505.4       505.4
                               DEC           50.5       50.5        50.5
Emission Intensity [gCO2/kWh]  BAU          167.6      155.2       149.2
                               SE            78.8       78.8        78.8
                               DEC            7.9        7.9         7.9
Coal Capacity Factor [%]       BAU           81.3       80.6        80.6
                               SE             6.9       25.3        30.8
                               DEC            0.0        1.8         1.8
Transmission Capacity [TVA km] BAU          388.5      405.4       432.5
                               SE           523.8      640.6       646.4
                               DEC          735.3     1379.4      1247.0
Energy Trade [TWh]             BAU          700.9     1344.8      1537.8
                               SE          1352.7     2528.6      2622.5
                               DEC         1541.2     5541.3      4854.4
Annual System Cost [EUR Bil/a] BAU          322.9      319.1       318.9
                               SE           342.2      332.3       330.3
                               DEC          646.2      385.3       374.0
Marginal Price [EUR/MWh]       BAU           50.3       49.8        49.7
                               SE            53.4       51.8        51.5
                               DEC          100.8       60.1        58.3

In [63]:
df_all.to_csv("Tables/KPI_summary.csv")

In [64]:
0.9193 ** 0.5

0.9588013350011565